#Mordern Portfolio theory

-- Markowitzn portfolio theory


In [ ]:
!pip install prophet
!pip install yfinance
!pip install pip install PyPortfolioOpt

In [53]:
#Import Libs
import yfinance as yf
import pandas as pd
from prophet import Prophet
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import risk_models
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [54]:
df = pd.read_csv('small_portfolio.csv',parse_dates=['date'],index_col='date')
df.head(2)

,GE,JPM,MSFT,PG
date,,,,
2015-01-02,25.06,62.49,46.760,90.44
2015-01-05,24.60,60.55,46.325,90.01


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 813 entries, 2015-01-02 to 2018-03-27
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GE      813 non-null    float64
 1   JPM     813 non-null    float64
 2   MSFT    813 non-null    float64
 3   PG      813 non-null    float64
dtypes: float64(4)
memory usage: 31.8 KB


In [56]:
from pypfopt import expected_returns
#Calculat expected annualized returns and sample covariance
mu = expected_returns.mean_historical_return(df)
sigma = risk_models.sample_cov(df)

In [85]:
#Obtain the EffecientFrontier
ef = EfficientFrontier(mu,sigma)

In [58]:
print(mu,sigma)

GE     -0.175812
JPM     0.185643
MSFT    0.223083
PG     -0.045684
dtype: float64             GE       JPM      MSFT        PG
GE    0.046355  0.023011  0.016415  0.010182
JPM   0.023011  0.047033  0.024328  0.010899
MSFT  0.016415  0.024328  0.054486  0.014257
PG    0.010182  0.010899  0.014257  0.020810


In [76]:
#Select a chosen Optimal portfolio
ef.max_sharpe()

OrderedDict([('GE', 0.0),
             ('JPM', 0.4251955192429594),
             ('MSFT', 0.5748044807570406),
             ('PG', 0.0)])

In [66]:
#select an optimal return for a target risk
ef.efficient_risk(2.5)

OrderedDict([('GE', 4.081159e-10),
             ('JPM', 4.202195e-09),
             ('MSFT', 0.9999999948273178),
             ('PG', 5.623253e-10)])

In [69]:
#Select an minimal risk for a target return
ef.efficient_return(0.2)

OrderedDict([('GE', 0.0),
             ('JPM', 0.4926529085352376),
             ('MSFT', 0.4900890447252643),
             ('PG', 0.017258046739498)])

In [70]:
#Calculate the Portfoli risk and performance

#obtain the performance numbers

ef.portfolio_performance(verbose=True,risk_free_rate=0.01)

Expected annual return: 20.0%
Annual volatility: 19.2%
Sharpe Ratio: 0.99


(0.19999999999999998, 0.19152666984849026, 0.9920289438035028)

#Maximum Sharpe vs minimum Volatility

In [72]:
#Maximun Sharpe Portfolio: the highest Sharpe Ratio no the EF

In [79]:
#Calculate the efficient Frontier with mu and sigma
raw_weights =ef.max_sharpe()

In [81]:
#Get interpolated Weights
cleaned_weights= ef.clean_weights()
cleaned_weights

OrderedDict([('GE', 0.0), ('JPM', 0.4252), ('MSFT', 0.5748), ('PG', 0.0)])

In [82]:
#get performance numbers
ef.portfolio_performance(verbose=True)

Expected annual return: 20.7%
Annual volatility: 19.6%
Sharpe Ratio: 0.96


(0.20716402573897588, 0.1959516973774276, 0.9551538886569299)

In [83]:
#Minumum Volatility Portfolio: the lowest level of risk on the effecient frontier

In [86]:
raw_weights_min= ef.min_volatility()

In [88]:
cleaned_weights = ef.clean_weights()
cleaned_weights

OrderedDict([('GE', 0.15732),
             ('JPM', 0.11714),
             ('MSFT', 0.04704),
             ('PG', 0.6785)])

In [94]:
#minimum Volatility Portfolio
min_vol_port = ef.portfolio_performance(verbose=True)

Expected annual return: -2.6%
Annual volatility: 13.3%
Sharpe Ratio: -0.35


#Alternative Portfolio Optimization

In [97]:
#Exponantially weighted Returns

#Exponantially weighted moving average

mu_ema =expected_returns.ema_historical_return(df,span=252,frequency=252)
print(mu_ema)

GE     -0.547256
JPM     0.229540
MSFT    0.321329
PG     -0.186417
Name: 2018-03-27 00:00:00, dtype: float64


In [99]:
#Exponentially weighted covariance
sigma_ew= risk_models.exp_cov(df, span=180, frequency= 252)
sigma_ew

,GE,JPM,MSFT,PG
GE,0.089302,0.026028,0.013672,0.009110
JPM,0.026028,0.045717,0.034885,0.009699
MSFT,0.013672,0.034885,0.067528,0.009651
PG,0.009110,0.009699,0.009651,0.022898


In [100]:
#Semicovariance in pyPortfolioOpt
Sigma_semi= risk_models.semicovariance(df,banchmark=0,frequency=252)
print(Sigma_semi)

            GE       JPM      MSFT        PG
GE    0.025319  0.014155  0.012131  0.008346
JPM   0.014155  0.021860  0.014094  0.008217
MSFT  0.012131  0.014094  0.023634  0.009140
PG    0.008346  0.008217  0.009140  0.011286
